In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 850M (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
path = "data/sign_mix_expedition3/"

In [3]:
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *

Using Theano backend.


In [4]:
batch_size=32

In [5]:
# This is for extracting out the convolutional part of vgg16
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [6]:
conv_model = Sequential(conv_layers)

In [7]:
# Generate batches for all our data
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False)

Found 1748 images belonging to 24 classes.
Found 336 images belonging to 24 classes.
Found 1 images belonging to 1 classes.


In [8]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 1748 images belonging to 24 classes.
Found 336 images belonging to 24 classes.
Found 1 images belonging to 1 classes.


# Memory expensive stuff coming up, execute only what's needed.

In [9]:
# Training data convolutional proccessing results
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
save_array(path+'results/conv_feat.dat', conv_feat)

In [10]:
# Validation data convolutional proccessing results
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
save_array(path+'results/conv_val_feat.dat', conv_val_feat)

In [11]:
# Testing data convolutional proccessing results
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [12]:
# Do only if really loading is needed
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_val_feat.shape

(336, 512, 14, 14)

# Fully connected model ahead

In [30]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/3),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(24, activation='softmax')
        ]

In [31]:
p=0.8

In [32]:
bn_model = Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Get augmented training batches
Do adjust data multiplication factor below

In [16]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.15, 
                                 channel_shift_range=20, width_shift_range=0.15)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

Found 1748 images belonging to 24 classes.


In [17]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*4)

In [18]:
save_array(path+'results/da_conv_feat2.dat', da_conv_feat)

In [ ]:
# Do only if needed
da_conv_feat = load_array(path+'results/da_conv_feat2.dat')

In [19]:
# Try this if it can be fitted into memory
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

In [20]:
da_trn_labels = np.concatenate([trn_labels]*5)

# Lets try fitting to this

In [22]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=10, 
             validation_data=(conv_val_feat, val_labels))

Train on 8740 samples, validate on 336 samples
Epoch 1/10
8740/8740 [==============================] - 10s - loss: 3.4515 - acc: 0.1554 - val_loss: 2.1684 - val_acc: 0.4911
Epoch 2/10
8740/8740 [==============================] - 10s - loss: 2.8098 - acc: 0.2238 - val_loss: 2.1259 - val_acc: 0.4940
Epoch 3/10
8740/8740 [==============================] - 10s - loss: 2.5016 - acc: 0.2870 - val_loss: 2.0800 - val_acc: 0.4821
Epoch 4/10
8740/8740 [==============================] - 10s - loss: 2.3087 - acc: 0.3347 - val_loss: 2.0458 - val_acc: 0.5298
Epoch 5/10
8740/8740 [==============================] - 10s - loss: 2.1957 - acc: 0.3632 - val_loss: 2.0169 - val_acc: 0.5417
Epoch 6/10
8740/8740 [==============================] - 10s - loss: 2.0857 - acc: 0.3970 - val_loss: 1.9750 - val_acc: 0.5804
Epoch 7/10
8740/8740 [==============================] - 10s - loss: 2.0196 - acc: 0.4207 - val_loss: 2.0153 - val_acc: 0.5119
Epoch 8/10
8740/8740 [==============================] - 10s - loss: 1.9

In [29]:
bn_model.save_weights(path+'models/da_conv8_1.h5')

In [33]:
# Do only when weights have been saved already
bn_model.load_weights(path+'models/da_conv8_1.h5')

# Lets test test test

In [25]:
test_batches = get_batches(path+'test', batch_size=batch_size, shuffle=False)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

Found 1 images belonging to 1 classes.


In [26]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [27]:
preds

array([[ 0.084 ,  0.0064,  0.0169,  0.011 ,  0.0614,  0.0073,  0.0159,  0.0165,  0.0089,  0.0138,
         0.0132,  0.0457,  0.0808,  0.0267,  0.0132,  0.0198,  0.0101,  0.1442,  0.2866,  0.0113,
         0.007 ,  0.0075,  0.0643,  0.0275]], dtype=float32)

In [20]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[1][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[1][0]            
___________________________________________________________________________________________

In [23]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=9, 
             validation_data=(conv_val_feat, val_labels))

Train on 8740 samples, validate on 336 samples
Epoch 1/9
8740/8740 [==============================] - 10s - loss: 1.7679 - acc: 0.4886 - val_loss: 2.0652 - val_acc: 0.5536
Epoch 2/9
8740/8740 [==============================] - 10s - loss: 1.7326 - acc: 0.5063 - val_loss: 2.1226 - val_acc: 0.5685
Epoch 3/9
8740/8740 [==============================] - 10s - loss: 1.6944 - acc: 0.5110 - val_loss: 2.1618 - val_acc: 0.5804
Epoch 4/9
8740/8740 [==============================] - 10s - loss: 1.6516 - acc: 0.5265 - val_loss: 2.2167 - val_acc: 0.5774
Epoch 5/9
8740/8740 [==============================] - 10s - loss: 1.6201 - acc: 0.5322 - val_loss: 2.2268 - val_acc: 0.5476
Epoch 6/9
8740/8740 [==============================] - 10s - loss: 1.6070 - acc: 0.5383 - val_loss: 2.2575 - val_acc: 0.5327
Epoch 7/9
8740/8740 [==============================] - 10s - loss: 1.5819 - acc: 0.5521 - val_loss: 2.3035 - val_acc: 0.5238
Epoch 8/9
8740/8740 [==============================] - 10s - loss: 1.5592 - ac

In [24]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=30, 
             validation_data=(conv_val_feat, val_labels))

Train on 8740 samples, validate on 336 samples
Epoch 1/30
8740/8740 [==============================] - 10s - loss: 1.5047 - acc: 0.5762 - val_loss: 2.3508 - val_acc: 0.5060
Epoch 2/30
8740/8740 [==============================] - 10s - loss: 1.4913 - acc: 0.5757 - val_loss: 2.3816 - val_acc: 0.5298
Epoch 3/30
8740/8740 [==============================] - 11s - loss: 1.4688 - acc: 0.5872 - val_loss: 2.4250 - val_acc: 0.5089
Epoch 4/30
8740/8740 [==============================] - 10s - loss: 1.4682 - acc: 0.5814 - val_loss: 2.4604 - val_acc: 0.4940
Epoch 5/30
8740/8740 [==============================] - 10s - loss: 1.4262 - acc: 0.6022 - val_loss: 2.4239 - val_acc: 0.4911
Epoch 6/30
8740/8740 [==============================] - 10s - loss: 1.4169 - acc: 0.6023 - val_loss: 2.4430 - val_acc: 0.4940
Epoch 7/30
8740/8740 [==============================] - 10s - loss: 1.3971 - acc: 0.6065 - val_loss: 2.4555 - val_acc: 0.4643
Epoch 8/30
8740/8740 [==============================] - 10s - loss: 1.3

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=50, 
             validation_data=(conv_val_feat, val_labels))